<a href="https://colab.research.google.com/github/petersonrs/petersonrs/blob/main/Koalas01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Caminhos dos dados

In [29]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
clientes = '/content/drive/MyDrive/Colab Notebooks/olist_customers_dataset.csv'
lista_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_items_dataset.csv'
pgto_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_payments_dataset.csv'
avaliacoes_pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_order_reviews_dataset.csv'
pedidos = '/content/drive/MyDrive/Colab Notebooks/olist_orders_dataset.csv'
produtos = '/content/drive/MyDrive/Colab Notebooks/olist_products_dataset.csv'
vendedores = '/content/drive/MyDrive/Colab Notebooks/olist_sellers_dataset.csv'
categoria_produtos = '/content/drive/MyDrive/Colab Notebooks/product_category_name_translation.csv'

# Caminho dos arquivos do Peterson

In [45]:
clientes = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_customers_dataset.csv'
lista_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_items_dataset.csv'
pgto_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_payments_dataset.csv'
avaliacoes_pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_order_reviews_dataset.csv'
pedidos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_orders_dataset.csv'
produtos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_products_dataset.csv'
vendedores = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/olist_sellers_dataset.csv'
categoria_produtos = '/content/drive/MyDrive/Colab Notebooks/StackKoala/cru/product_category_name_translation.csv'

## Import libs

In [46]:
import pandas as pd
import numpy as np

## lendo os csvs

In [47]:
ds_lista_pedidos = pd.read_csv(lista_pedidos, index_col=False, encoding='utf8')
ds_pgto_pedidos = pd.read_csv(pgto_pedidos, index_col=False, encoding='utf8')
ds_avaliacoes_pedidos = pd.read_csv(avaliacoes_pedidos, index_col=False, encoding='utf8')
ds_pedidos = pd.read_csv(pedidos, index_col=False, encoding='utf8')
ds_produtos = pd.read_csv(produtos, index_col=False, encoding='utf8')
ds_vendedores = pd.read_csv(vendedores, index_col=False, encoding='utf8')
ds_clientes = pd.read_csv(clientes, index_col=False, encoding='utf8')
ds_categoria_produtos = pd.read_csv(categoria_produtos, index_col=False, encoding='utf8')

## join entre as tabelas para montar um tabelão

In [48]:
result = ds_pedidos.merge(ds_lista_pedidos, on="order_id", how="left")

In [49]:
result = result.merge(ds_produtos, on='product_id', how='left')

In [50]:
result = result.merge(ds_vendedores, on='seller_id', how='left')

In [51]:
result = result.merge(ds_clientes, on='customer_id', how='left')

In [52]:
result = result.merge(ds_avaliacoes_pedidos, on='order_id', how='left')

In [53]:
result = result.merge(ds_pgto_pedidos, on='order_id', how='left')

## exibindo como ficaram junções

In [54]:
result.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2.0,voucher,1.0,18.59
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,boleto,1.0,141.46
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,1.0,credit_card,3.0,179.12


## backup do resultado e store na pasta de dados processados

In [55]:
transformacao_final = result.copy()

In [ ]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/dados_transformados.csv',  index=False, encoding='utf8')

## Copia do arquivo processado na sessao do Peterson

In [56]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados.csv',  index=False, encoding='utf8')

In [57]:
transformacao_final.head(2)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00


In [ ]:
## Transformando *.csv em *.parquet
transformacao_final.to_parquet('/content/drive/MyDrive/Colab Notebooks/dados_transformados.parquet', index=False)

In [ ]:
#transformacao_final = transformacao_final.drop(columns=['review_comment_title'], axis=1)
#transformacao_final.head(5)

## Testando review_score
## 1-2 Ruim ou Péssimo
## 4-5 Bom ou Ótimo
## 3 Neutro

Adicionando uma coluna nova para receber novos valores de titulo

In [97]:
transformacao_final['new_title_review'] = None

In [98]:
def TestReviewScoreTitle():
  for index, row in transformacao_final.iterrows():
    if (transformacao_final['review_score'][index] >= 4 and transformacao_final['review_comment_title'][index] is np.nan):
        transformacao_final['new_title_review'][index] = 'bom ou otimo'
    elif (transformacao_final['review_score'][index] < 3 and transformacao_final['review_comment_title'][index] is np.nan):
          transformacao_final['new_title_review'][index] = 'ruim ou pessimo'
    elif (transformacao_final['review_score'][index] == 3 and transformacao_final['review_comment_title'][index] is np.nan):
          transformacao_final['new_title_review'][index] = 'neutro'
    else:
      transformacao_final['new_title_review'][index] = transformacao_final['review_comment_title'][index]


In [99]:
TestReviewScoreTitle()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/py

In [103]:
transformacao_final.head(5)

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,seller_zip_code_prefix,seller_city,seller_state,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,new_title_review,new_message_review
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12,bom ou otimo,None
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,3.0,voucher,1.0,2.00,bom ou otimo,None
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,maua,SP,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11 00:00:00,2017-10-12 03:43:48,2.0,voucher,1.0,18.59,bom ou otimo,None
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,belo horizonte,SP,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08 00:00:00,2018-08-08 18:37:50,1.0,boleto,1.0,141.46,Muito boa a loja,None
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,guariba,SP,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18 00:00:00,2018-08-22 19:07:58,1.0,credit_card,3.0,179.12,bom ou otimo,None


## Ajustando as mensagens nulas

In [104]:
transformacao_final['new_message_review'] = None

In [105]:
def TestReviewScoreMessages():
  for index, row in transformacao_final.iterrows():
    if (transformacao_final['review_score'][index] >= 4 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'bom ou otimo'
    elif (transformacao_final['review_score'][index] < 3 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'ruim ou pessimo'
    elif (transformacao_final['review_score'][index] == 3 and transformacao_final['review_comment_message'][index] is np.nan):
        transformacao_final['new_message_review'][index] = 'neutro'
    else:
      transformacao_final['new_message_review'][index] = transformacao_final['review_comment_message'][index]

In [ ]:
TestReviewScoreMessages()

In [ ]:
transformacao_final.head(50)

# Salvando arquivo com textos processados.
Agora pode ler a partir deste arquivo.

In [108]:
transformacao_final.to_csv('/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_tratado.csv',  index=False, encoding='utf8')

#A partir daqui Novos tratamentos no arquivo
#Tratar parte nlp nos titulos e mensagens

In [ ]:
dados_tratados = '/content/drive/MyDrive/Colab Notebooks/StackKoala/processado/dados_transformados_tratado.csv'
ds_dados_tratados = pd.read_csv(dados_tratados, index_col=False, encoding='utf8')
transformacao_final = ds_dados_tratados.copy() # só pra manter o mesmo nome


## tratando apenas reviews